# Enviornment setup

The goal of this first partical is to setup the environment.
To start, we will use colab to be sure you can execute notebooks and complete future practicals.
Then we will go through the installation of a Python environment on your local machine.

## Install dependencies (Colab Only)

Installing dependencies allows you to reuse tools, libraries and framework created by others.
The most used dependencies manager in Python is pip. Pip is a command line tool that pull dependencies from a repository . The repository is often the centralized https://pypi.org/, but can also be the private repository of a company.


In the next cell, we will install common dependencies that you will need in future practicals. The list of dependencies is located in `requirements.txt` by convention. (Focus the cell and execute it by pressing `ctrl + enter`).

In [ ]:
%pip install -r /content/drive/MyDrive/cyberus_ml_security/0_environment_setup/requirements.txt

## Import package

It is good practice to import all necessary packages at the top of Python files or in the first code cell of a Python notebook.

In [ ]:
import torch
import sklearn
import mlc
from mlc.datasets.dataset_factory import get_dataset
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

We check the correct version are installed.

In [ ]:
for pkg, version in [(torch, "1.12.1"), (sklearn, "1.2.1"), (mlc, "0.1.0")]:
    if version in pkg.__version__:
        print(f"OK: {pkg.__name__}=={pkg.__version__}.")
    else:
        print(f"Version mismatch: expected version {version} for package {pkg.__name__} but is currently {pkg.__version__}")

## Retrieve data

In this section we will download and load a feature engineered version of the popular Lending Club Loan Data dataset ([LCLD](https://www.kaggle.com/datasets/wordsforthewise/lending-club)).

In [ ]:
dataset = get_dataset("lcld_v2_iid")
x, y = dataset.get_x_y()
metadata = dataset.get_metadata(only_x=True)

In [ ]:
# Splitting the data
splits = dataset.get_splits()
x_train, x_val, x_test = x.iloc[splits["train"]], x.iloc[splits["val"]], x.iloc[splits["test"]]
y_train, y_val, y_test = y[splits["train"]], y[splits["val"]], y[splits["test"]]


In [ ]:
# Scaling the data, the metadata contains for each feature, its name (feature) and its type (type).
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), metadata[metadata["type"] != "cat"]["feature"]),
        ('cat', OneHotEncoder(), metadata[metadata["type"] == "cat"]["feature"])
    ])

preprocessor.fit(x)
x_train = preprocessor.transform(x_train)
x_val =  preprocessor.transform(x_val)
x_test =  preprocessor.transform(x_test)

## Fit sklearn model

In [ ]:
model = RandomForestClassifier(n_estimators=10, class_weight="balanced", n_jobs=-1, verbose=2)
model.fit(x_train, y_train)

In [ ]:
# Model prediction
y_score = model.predict_proba(x_test)


In [ ]:
# Model scoring
auc = roc_auc_score(y_test, y_score[:, 1])
print(f"The AUROC score of the model is {auc}")

In [ ]:
x_train.shape

## Fit torch Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(50, 64)
        self.l2 = nn.Linear(64, 32)
        self.l3 = nn.Linear(32, 16)
        self.l4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        return x



In [ ]:
class_weight = torch.Tensor(
    1 - torch.unique(torch.tensor(y_train), return_counts=True)[1] / len(y_train)
)
print(f"Class weight {class_weight}")

In [ ]:
model = Net()
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.01,
)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = len(dataloader.dataset)
    for batch, (X, y) in tqdm(enumerate(dataloader), total=int(size/batch_size)):
        # if batch % 10 == 0:
        #     print(f"Batch {batch}.")
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def val_loop(dataloader, model, loss_fn, epoch_i):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y[:, 1]).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Epoch {epoch_i}, Val Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")



def train_model(model, x_train, y_train, x_val, y_val, optimizer, batch_size, loss_func, epochs):
    # Data processing
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
    )
    val_dataset = TensorDataset(x_val, y_val)
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=2000,
        shuffle=True,
        num_workers=2,
    )

    # Main train loop
    for epoch in range(epochs):
        train_loop(train_loader, model, loss_func, optimizer, batch_size)
        val_loop(val_loader, model, loss_func, epoch)




In [ ]:
loss = nn.CrossEntropyLoss(weight=class_weight)
train_model(
    model,
    torch.from_numpy(x_train).float(),
    torch.from_numpy(np.array([1 - y_train, y_train]).T).float(),
    torch.from_numpy(x_val).float(),
    torch.from_numpy(np.array([1 - y_val, y_val]).T).float(),
    optimizer,
    1024,
    loss,
    2
)

In [ ]:
# Model prediction
y_score = model(torch.from_numpy(x_test).float()).detach().numpy()


In [ ]:
# Model scoring
auc = roc_auc_score(y_test, y_score[:, 1])
print(f"The AUROC score of the model is {auc}")

## Future practical

If you reach this section without trouble, you should be able to complete future practicals.

The list of dependencies in the requirements.txt file may be updated with each practical. If some packages do not import properly, your dependencies are probably outdated. Simply re-run `pip install -r requirements.txt`. If you are using a local environment, it is recommended to create a new conda environment for each practical.

If any questions remain to setup your environment, do not hesitate to contact the teaching team.